In [13]:
# csv to sqlite database
# import dependencies
import pandas as pd
from sqlalchemy import Column, String, Integer, Float, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, MetaData, Table, inspect 
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker

In [14]:
# importing the csv files for article counts and google trends 
path1 = "Guard_df.csv"
path2 = "NYT_df.csv"
path3 = "google_olympics_worldwide-multiTimeline.csv"
Guard_df = pd.read_csv(path1)
NYT_df = pd.read_csv(path2)
Googl_df = pd.read_csv(path3, header=1)

In [15]:
# changing google data to datetime, just in case
''' Messes up later with plotting. Since only contains year and month, 
it artificially adds the first of the month as the day.

Googl_df["Month"] = pd.to_datetime(Googl_df["Month"])
Googl_df.dtypes'''

' Messes up later with plotting. Since only contains year and month, \nit artificially adds the first of the month as the day.\n\nGoogl_df["Month"] = pd.to_datetime(Googl_df["Month"])\nGoogl_df.dtypes'

In [16]:
# adding olympics_year data ("olympics" if summer or regular olympics, "winter" if winter olympics)
# data starts at 1918
# ToDo if necessary

### Creating Sqlite database

In [17]:
# *** Using Declarative Base
Base = declarative_base()


In [18]:
# creating NYT table for the database
class NYT(Base):
    __tablename__ = "NYT_api_results"
    news_id = Column(Integer, primary_key=True, nullable=False)
    year = Column(Integer)
    article_hits = Column(Integer)


In [19]:
# theGuardian table for the database
class Guard(Base):
    __tablename__ = "Guard_api_results"
    news_id = Column(Integer, primary_key=True, nullable=False)
    year = Column(Integer)
    article_hits = Column(Integer)

In [20]:
# Google Trends data for the database
class GooglTrends(Base):
    __tablename__ = "Google_trends_results"
    goog_id = Column(Integer, primary_key=True, nullable=False, autoincrement=True)
    month = Column(String(32))
    trend = Column(Integer)

In [21]:
# Use the `engine` and connection string to create a database called `newsAPI.sqlite`
engine = create_engine('sqlite:///newsAPI.sqlite')
# drop tables if exist
if engine.dialect.has_table:
    print("dropping all tables!")
    Base.metadata.drop_all(engine)
    
conn = engine.connect()
# Create the tables within the database
Base.metadata.create_all(engine)

# create session
session = Session(bind=engine)


dropping all tables!


In [22]:
# Adding NYT data

for index, row in NYT_df.iterrows():
    record = NYT(news_id = int(row['Unnamed: 0']), year = int(row["Year"]),\
                 article_hits = int(row["article_hits"]))
    session.add(record)
    session.commit()
    
print("finished adding NYT records!")


finished adding NYT records!


In [23]:
# Adding Guardian data
for index, row in Guard_df.iterrows():
    record = Guard(news_id = int(row["Unnamed: 0"]),\
                   year = int(row["Year"]), article_hits = int(row["article_hits"]))
    session.add(record)
    session.commit()
    
print("finished adding Guardian record!")


finished adding Guardian record!


In [24]:
# Adding Google Trends data
for index, row in Googl_df.iterrows():
    record = GooglTrends(month = row["Month"],\
                         trend = int(row["olympics: (Worldwide)"]))
    session.add(record)
    session.commit()
    
print("finished adding Google Trends record!")

finished adding Google Trends record!


In [25]:
# checking to see if data is added 
query = "SELECT * FROM Google_trends_results"
pd.read_sql_query(query, engine).head()

,goog_id,month,trend
0,1,2004-01,3
1,2,2004-02,4
2,3,2004-03,4
3,4,2004-04,4
4,5,2004-05,5
